In [1]:
import simpy
import random
import pandas as pd

In [91]:
class Consola:
    def __init__(self, env, id, tipo, juegos_disponibles):
        self.env = env
        self.id = id
        self.tipo = tipo
        self.juegos_disponibles = juegos_disponibles
        self.recurso = simpy.Resource(env, capacity=1)  # Solo 1 cliente por consola

In [ ]:
class SistemaAlquiler:
    def __init__(self, env, num_consolas_xbox, num_consolas_ps):
        self.env = env
        self.recursos = {
            'ps': [Consola(env, f'ps_{i+1}', 'ps', ['fifa', 'call_of_duty']) for i in range(num_consolas_ps)],
            'xbox': [Consola(env, f'xbox_{i+1}', 'xbox', ['fifa', 'call_of_duty']) for i in range(num_consolas_xbox)]
        }
        self.clientes = []

    def agregar_cliente(self, tipo_consola, juego, paciente):
        cliente = Cliente(self.env, len(self.clientes)+1, self, tipo_consola, juego, paciente)
        self.clientes.append(cliente)


In [92]:

class Cliente:
    def __init__(self, env, id, sistema, tipo_consola, juego, paciente):
        self.env = env
        self.id = id
        self.sistema = sistema
        self.tipo_consola = tipo_consola
        self.juego = juego
        self.paciente = paciente
        self.eventos = pd.DataFrame()
        self.costo = 0  # Costo inicial
        self.atendido = False  # Flag para saber si fue atendido
        self.env.process(self.proceso())

    def registrar_evento(self, evento):
        print(f"{self.env.now}: Cliente {self.id} - {evento}")
        self.eventos = pd.concat([self.eventos, pd.DataFrame({
            "id": [self.id],
            "evento": [evento],
            "time": [self.env.now]
        })])

    def proceso(self):
        self.registrar_evento(f"LLEGA - Quiere jugar en la consola {self.tipo_consola} con el juego {self.juego}")
        
        # Verificación de consolas disponibles
        disponible = False
        consolas = self.sistema.recursos[self.tipo_consola]

        for consola in consolas:
            if consola.recurso.count < consola.recurso.capacity:
                # Si la consola está disponible
                disponible = consola
                break
        
        if disponible:
            self.registrar_evento(f"Consola {disponible.id} ESTÁ DISPONIBLE para {self.tipo_consola} con el juego {self.juego}")
            
            with disponible.recurso.request() as req:
                yield req
                self.registrar_evento(f"COMIENZA JUEGO en {disponible.id}")
                tiempo_juego = 60  # 1 hora de juego
                yield self.env.timeout(tiempo_juego)
                self.registrar_evento(f"TERMINA JUEGO en {disponible.id}")
                
                # El cliente paga la hora que jugó
                self.costo += 1  # Pago por una hora de juego
                self.registrar_evento(f"Cliente {self.id} PAGA 1 HORA")
                
                # Decisión de si el cliente se queda otra hora
                if self.paciente and random.choice([True, False]):  # Aleatorio si decide quedarse
                    self.registrar_evento(f"Cliente {self.id} DECIDE QUEDARSE UNA HORA MÁS")
                    yield self.env.timeout(60)  # Tiempo adicional de 1 hora
                    self.costo += 1  # Agregar el pago por la segunda hora
                    self.registrar_evento(f"Cliente {self.id} PAGA OTRA HORA. TOTAL = {self.costo} HORA(S)")

                # Marcar como atendido
                self.atendido = True
        else:
            self.registrar_evento(f"SE VA POR FALTA DE CONSOLA DISPONIBLE - {self.tipo_consola} con el juego {self.juego}")


In [134]:
# Simulación
env = simpy.Environment()

# Puedes cambiar la cantidad de consolas en este punto
num_consolas_xbox = 2  # Número de consolas Xbox
num_consolas_ps = 2  # Número de consolas PS

# Crear el sistema de alquiler con las consolas dinámicas
sistema = SistemaAlquiler(env, num_consolas_xbox, num_consolas_ps)
random.seed(42)

In [135]:
# Crear el proceso de nuevos clientes que llegan aleatoriamente
def crear_nuevos_clientes(env, sistema):
    while True:
        yield env.timeout(random.randint(1, 5))  # Los clientes llegan cada entre 0 y 5 minutos

        # Generar aleatoriamente el tipo de consola y juego
        tipo_consola = random.choice(['ps', 'xbox'])
        if tipo_consola == 'ps':
            juego = random.choice(['fifa', 'call_of_duty'])  # Aleatorio para PS
        else:
            juego = random.choice(['fifa', 'call_of_duty'])  # Aleatorio para Xbox
        
        paciente = random.choice([True, False])  # Puede esperar o no
        
        sistema.agregar_cliente(tipo_consola, juego, paciente)

# Iniciar el proceso de nuevos clientes
env.process(crear_nuevos_clientes(env, sistema))

# Correr simulación durante 12 horas (720 minutos)
env.run(until=60 * 12)

1: Cliente 1 - LLEGA - Quiere jugar en la consola ps con el juego call_of_duty
1: Cliente 1 - Consola ps_1 ESTÁ DISPONIBLE para ps con el juego call_of_duty
1: Cliente 1 - COMIENZA JUEGO en ps_1
3: Cliente 2 - LLEGA - Quiere jugar en la consola ps con el juego fifa
3: Cliente 2 - Consola ps_2 ESTÁ DISPONIBLE para ps con el juego fifa
3: Cliente 2 - COMIENZA JUEGO en ps_2
8: Cliente 3 - LLEGA - Quiere jugar en la consola xbox con el juego fifa
8: Cliente 3 - Consola xbox_1 ESTÁ DISPONIBLE para xbox con el juego fifa
8: Cliente 3 - COMIENZA JUEGO en xbox_1
9: Cliente 4 - LLEGA - Quiere jugar en la consola ps con el juego fifa
9: Cliente 4 - SE VA POR FALTA DE CONSOLA DISPONIBLE - ps con el juego fifa
14: Cliente 5 - LLEGA - Quiere jugar en la consola ps con el juego call_of_duty
14: Cliente 5 - SE VA POR FALTA DE CONSOLA DISPONIBLE - ps con el juego call_of_duty
18: Cliente 6 - LLEGA - Quiere jugar en la consola xbox con el juego fifa
18: Cliente 6 - Consola xbox_2 ESTÁ DISPONIBLE para x

### DATAFRAME DEL CLIENTE

In [124]:
# Crear el perfil del cliente
lista_clientes = []
for c in sistema.clientes:
    lista_clientes.append({
        "id": c.id,
        "tipo_consola": c.tipo_consola,
        "juego": c.juego,
        "paciente": c.paciente,
        "atendido": c.atendido,
        "costo": c.costo
    })
dfclientes = pd.DataFrame(lista_clientes)

# Mostrar el perfil de un cliente específico
dfclientes.loc[dfclientes["id"] == 2, :]


,id,tipo_consola,juego,paciente,atendido,costo
1,2,ps,fifa,True,True,1


### DATAFRAME DE LOS EVENTOS

In [117]:
# Crear el DataFrame de eventos
lista_eventos = []
for c in sistema.clientes:
    lista_eventos.append(c.eventos)
dfeventos = pd.concat(lista_eventos)
dfeventos = dfeventos.reset_index()
dfeventos.drop(["index"], axis=1, inplace=True)
dfeventos


,id,evento,time
0,1,LLEGA - Quiere jugar en la consola ps con el j...,1
1,1,Consola ps_1 ESTÁ DISPONIBLE para ps con el ju...,1
2,1,COMIENZA JUEGO en ps_1,1
3,1,TERMINA JUEGO en ps_1,61
4,1,Cliente 1 PAGA 1 HORA,61
...,...,...,...
590,233,LLEGA - Quiere jugar en la consola ps con el j...,715
591,233,Consola ps_2 ESTÁ DISPONIBLE para ps con el ju...,715
592,233,COMIENZA JUEGO en ps_2,715
593,234,LLEGA - Quiere jugar en la consola ps con el j...,718


#### 1. ¿Cuál fue la consola mas utilizada?

In [118]:
dfclientes["tipo_consola"].value_counts()


tipo_consola
xbox    126
ps      108
Name: count, dtype: int64

#### 2. ¿Cuantas personas decidieron quedarse una hora más?¿Cuál es el probabilidad de que un cliente se quede 1h más?

In [119]:
dfeventos[dfeventos["evento"].str.contains("DECIDE QUEDARSE UNA HORA MÁS")]["id"].nunique()


10

In [120]:
clientes_quedaron_mas = dfeventos[dfeventos["evento"].str.contains("DECIDE QUEDARSE UNA HORA MÁS")]["id"].nunique()
clientes_atendidos = dfclientes[dfclientes["atendido"] == True]["id"].nunique()

probabilidad_quedarse = clientes_quedaron_mas / clientes_atendidos
print(f"Probabilidad de que un cliente atendido se quede una hora más: {probabilidad_quedarse:.2%}")


Probabilidad de que un cliente atendido se quede una hora más: 29.41%


#### 3. ¿Cuál fue el juego más utilizado en cada tipo de consola?

In [121]:
dfclientes.groupby(["tipo_consola", "juego"])["id"].count().reset_index(name="uso_total").sort_values(["tipo_consola", "uso_total"], ascending=[True, False])


,tipo_consola,juego,uso_total
0,ps,call_of_duty,55
1,ps,fifa,53
2,xbox,call_of_duty,74
3,xbox,fifa,52


#### 4. ¿Qué porcentaje de los clientes fueron atendidos?

In [122]:
atendidos = dfclientes["atendido"].sum()
total_clientes = dfclientes.shape[0]
porcentaje_atendidos = (atendidos / total_clientes) * 100

print(f"Porcentaje de clientes atendidos: {porcentaje_atendidos:.2f}%")


Porcentaje de clientes atendidos: 14.53%


#### 5. ¿Qué consola tuvo la mayor cantidad de sesiones extendidas?

In [123]:
df_extra = dfeventos[dfeventos["evento"].str.contains("DECIDE QUEDARSE UNA HORA MÁS")]
df_extra.merge(dfclientes, on="id").groupby("tipo_consola")["id"].count().reset_index(name="extensiones")


,tipo_consola,extensiones
0,ps,2
1,xbox,8


#### 6. ¿Cuantos clientes no fueron atendidos?

In [129]:
total_clientes = dfclientes["id"].nunique()
print(f"Total de clientes: {total_clientes}")

Total de clientes: 234


In [130]:
clientes_atendidos = dfclientes[dfclientes["atendido"] == True]["id"].nunique()
print(f"Clientes atendidos: {clientes_atendidos}")

Clientes atendidos: 34


In [ ]:
clientes_no_atendidos = total_clientes - clientes_atendidos

probabilidad_no_ser_atendido = clientes_no_atendidos / total_clientes

print(f"Cantidad de clientes que NO fueron atendidos: {clientes_no_atendidos}")
print(f"Probabilidad de que un cliente NO sea atendido: {probabilidad_no_ser_atendido:.2%}")


Cantidad de clientes que NO fueron atendidos: 175
Probabilidad de que un cliente NO sea atendido: 74.15%
